# Train Pytorch Explanation based Model for Fashion MNIST

# ref: tensorflow version from http://localhost:8888/notebooks/xai-adv/train_exp_based_model_v1.4_FMNIST_defender.ipynb

#### difference from FMNIST training is the number of epochs

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils import data as D
import os
import numpy as np
import numpy as np
from keras import datasets
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from PIL import Image

Using TensorFlow backend.


In [2]:
class Net(nn.Module):
    def __init__(self, features, num_classes, init_weights=True):
        super(Net, self).__init__()
        
        self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(4*4*50, 500),
            nn.ReLU(True),
            nn.Linear(500, num_classes)
        )
        
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        
        # x are the logits values
        return x 
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        
        output = F.log_softmax(output, dim=1)
        
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            output = F.log_softmax(output, dim=1)
            
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [3]:
"""
torch.nn.Conv2d(in_channels, out_channels, kernel_size, 
stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
"""

def make_layers(cfg, in_channels, kernel_size, stride, padding, batch_norm=False):
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=kernel_size, padding=padding)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


"""
Refer VGG19_bn configurationh here: 
https://github.com/pytorch/vision/blob/76702a03d6cc2e4f431bfd1914d5e301c07bd489/torchvision/models/vgg.py#L63
"""
cfgs = {
    #'E': [64, 64, 'M',128, 128, 'M',256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M',512, 512, 512, 512, 'M'],
    'E': [20, 'M', 50, 'M']
}

model_layers = make_layers(cfgs['E'],in_channels=1, kernel_size=5, stride=1, padding=0, batch_norm=False)

In [4]:
model_layers

Sequential(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [5]:
class CustomDS(D.Dataset):
    """
    A customized data loader.
    """
    def __init__(self, path, train=True):
        """ Intialize the dataset
        """
        if train:
            data_path = os.path.join(path,'x_train.npy')
            targets_path = os.path.join(path,'y_train.npy')
        else:
            data_path = os.path.join(path,'x_test.npy')
            targets_path = os.path.join(path,'y_test.npy')
        self.path = data_path
        self.data = np.load(data_path)
        self.targets = np.load(targets_path)
        #self.transform = transforms.ToTensor()
        self.transform = transforms.Compose([
                       transforms.ToTensor()
                       #transforms.Normalize((0.1307,), (0.3081,))
                   ])
        self.len = np.shape(self.data)[0]
        
    # You must override __getitem__ and __len__
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        data = self.data[index]
        image = Image.fromarray(data)
        
        target = int(self.targets[index])
        
        #data = (data * 255).astype(np.uint8)
        #data = data.reshape(28,28)
        #image = Image.fromarray((data * 255).astype(np.uint8))
        #image = Image.fromarray(data.astype(np.uint8))
        
        return self.transform(image), target

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

In [6]:
IS_DATA_READY = False

if not IS_DATA_READY:
    
    class_ind = 9
    directory = 'data/defender/fmnist/exp_model/for_target/' + str(class_ind)
    clean_intr_train_dir = 'data/defender/fmnist/intr_of_clean_train_bytarget/' + str(class_ind)

    ox = np.load(clean_intr_train_dir + '/cleaned_intr_of_x_train.npy')
    ax = np.load(directory + '/intr_of_adv_with_target_'  + str(class_ind) + '.npy')

    print('orig {} : {}'.format(class_ind, ox.shape))
    print('adv {} : {}'.format(class_ind, ax.shape))
    print('\n')
    ax = np.vstack((ax,ax))
    ax = np.vstack((ax,ax))
    ax = np.vstack((ax,ax))
    ax = np.vstack((ax,ax))

    ax = ax[:ox.shape[0]]

    # shuffle ax
    indices = np.arange(ax.shape[0])
    np.random.shuffle(indices)
    ax = ax[indices]

    class_0 = ox
    class_1 = ax

    total_x = np.concatenate((class_0, class_1))
    total_x = total_x.reshape(-1,28,28)

    print('total shape {}'.format(total_x.shape))

    y_0 = np.zeros((len(class_0)))
    y_1 = np.ones((len(class_1)))

    total_y = np.concatenate((y_0, y_1))
    
    #train, test split
    x_train, x_test, y_train, y_test = train_test_split(total_x,total_y,test_size=0.20, random_state=42, shuffle=True)

    #train, valid split from the remaining train set (after test set has been taken out)
    x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train,test_size=0.20, random_state=42, shuffle=True)

    
    print('x_train shape : {}'.format(x_train.shape))
    print('y_train shape : {}'.format(y_train.shape))
    print('x_test shape : {}'.format(x_test.shape))
    print('y_test shape : {}'.format(y_test.shape))
    print('x_valid shape : {}'.format(x_valid.shape))
    print('y_valid shape : {}'.format(y_valid.shape))
    
    directory = 'data/defender/fmnist/exp_model_data/for_target/' + str(class_ind)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    np.save(directory + '/x_train.npy', x_train)
    np.save(directory + '/y_train.npy', y_train)
    np.save(directory + '/x_test.npy', x_test)
    np.save(directory + '/y_test.npy', y_test)
    np.save(directory + '/x_valid.npy', x_valid)
    np.save(directory + '/y_valid.npy', y_valid)

else:
    class_ind = 6
    directory = 'data/defender/fmnist/exp_model_data/for_target/' + str(class_ind)
    
    x_train = np.load(directory + '/x_train.npy')
    y_train = np.load(directory + '/y_train.npy')
    x_test = np.load(directory + '/x_test.npy')
    y_test = np.load(directory + '/y_test.npy')
    x_valid = np.load(directory + '/x_valid.npy')
    y_valid = np.load(directory + '/x_valid.npy')
    
    print('x_train shape : {}'.format(x_train.shape))
    print('y_train shape : {}'.format(y_train.shape))
    print('x_test shape : {}'.format(x_test.shape))
    print('y_test shape : {}'.format(y_test.shape))
    print('x_valid shape : {}'.format(x_valid.shape))
    print('y_valid shape : {}'.format(y_valid.shape))  

orig 9 : (5988, 28, 28, 1)
adv 9 : (3126, 28, 28, 1)


total shape (11976, 28, 28)
x_train shape : (7664, 28, 28)
y_train shape : (7664,)
x_test shape : (2396, 28, 28)
y_test shape : (2396,)
x_valid shape : (1916, 28, 28)
y_valid shape : (1916,)


In [7]:
print(np.min(x_train[0]))
print(np.max(x_train[0]))

0.0
1.0


In [8]:
# Simple dataset. Only save path to image and load it and transform to tensor when call __getitem__.
filepath = directory
train_set = CustomDS(filepath, train=True)
test_set = CustomDS(filepath, train=False)

# total images in set
print(train_set.len)
print(test_set.len)

7664
2396


In [9]:
# main method
## Training settings
# input batch size for training (default: 64)
BATCH_SIZE = 64

# input batch size for testing (default: 1000)
TEST_BATCH_SIZE = 1000

# number of epochs to train
EPOCHS = 50

#learning rate (default: 0.01)
LR = 0.01

#SGD momentum (default: 0.5)
MOMENTUM = 0.5

# how many batches to wait before logging training status
LOG_INTERVAL = 10

SAVE_MODEL = True
SEED = 1
NO_CUDA = False
USE_CUDA = not NO_CUDA and torch.cuda.is_available()

NUM_CLASSES=2 # we are training a binary model (0 for normal or 1 for abnormal)

torch.manual_seed(SEED)

device = torch.device("cuda" if USE_CUDA else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if USE_CUDA else {}


In [10]:
model = Net(model_layers, num_classes=NUM_CLASSES).to(device)

In [11]:
model

Net(
  (features): Sequential(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=800, out_features=500, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=500, out_features=2, bias=True)
  )
)

In [12]:
# Setup train and test loader

train_loader = D.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False, **kwargs)
test_loader = D.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, **kwargs)


optimizer = optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM)

for epoch in range(1, EPOCHS + 1):
    train(model, device, train_loader, optimizer, epoch, LOG_INTERVAL)
    test(model, device, test_loader)

model_dir = directory + '/model'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
if (SAVE_MODEL):
    print('saving model to ', model_dir+"/exp_model.pt")
    torch.save(model.state_dict(),model_dir+"/exp_model.pt")

Train Epoch: 1 [0/7664 (0%)]	Loss: 0.693162
Train Epoch: 1 [640/7664 (8%)]	Loss: 0.692534
Train Epoch: 1 [1280/7664 (17%)]	Loss: 0.693453
Train Epoch: 1 [1920/7664 (25%)]	Loss: 0.692236
Train Epoch: 1 [2560/7664 (33%)]	Loss: 0.693346
Train Epoch: 1 [3200/7664 (42%)]	Loss: 0.692453
Train Epoch: 1 [3840/7664 (50%)]	Loss: 0.691006
Train Epoch: 1 [4480/7664 (58%)]	Loss: 0.691412
Train Epoch: 1 [5120/7664 (67%)]	Loss: 0.692710
Train Epoch: 1 [5760/7664 (75%)]	Loss: 0.691222
Train Epoch: 1 [6400/7664 (83%)]	Loss: 0.691988
Train Epoch: 1 [7040/7664 (92%)]	Loss: 0.688893

Test set: Average loss: 0.6915, Accuracy: 1167/2396 (49%)

Train Epoch: 2 [0/7664 (0%)]	Loss: 0.692775
Train Epoch: 2 [640/7664 (8%)]	Loss: 0.689672
Train Epoch: 2 [1280/7664 (17%)]	Loss: 0.694326
Train Epoch: 2 [1920/7664 (25%)]	Loss: 0.691359
Train Epoch: 2 [2560/7664 (33%)]	Loss: 0.690614
Train Epoch: 2 [3200/7664 (42%)]	Loss: 0.690314
Train Epoch: 2 [3840/7664 (50%)]	Loss: 0.687412
Train Epoch: 2 [4480/7664 (58%)]	Loss: 0

Train Epoch: 14 [0/7664 (0%)]	Loss: 0.373515
Train Epoch: 14 [640/7664 (8%)]	Loss: 0.486956
Train Epoch: 14 [1280/7664 (17%)]	Loss: 0.390034
Train Epoch: 14 [1920/7664 (25%)]	Loss: 0.331048
Train Epoch: 14 [2560/7664 (33%)]	Loss: 0.426113
Train Epoch: 14 [3200/7664 (42%)]	Loss: 0.518662
Train Epoch: 14 [3840/7664 (50%)]	Loss: 0.424618
Train Epoch: 14 [4480/7664 (58%)]	Loss: 0.294190
Train Epoch: 14 [5120/7664 (67%)]	Loss: 0.467874
Train Epoch: 14 [5760/7664 (75%)]	Loss: 0.359591
Train Epoch: 14 [6400/7664 (83%)]	Loss: 0.368960
Train Epoch: 14 [7040/7664 (92%)]	Loss: 0.475899

Test set: Average loss: 0.3908, Accuracy: 1942/2396 (81%)

Train Epoch: 15 [0/7664 (0%)]	Loss: 0.360947
Train Epoch: 15 [640/7664 (8%)]	Loss: 0.474717
Train Epoch: 15 [1280/7664 (17%)]	Loss: 0.383665
Train Epoch: 15 [1920/7664 (25%)]	Loss: 0.312388
Train Epoch: 15 [2560/7664 (33%)]	Loss: 0.404184
Train Epoch: 15 [3200/7664 (42%)]	Loss: 0.512873
Train Epoch: 15 [3840/7664 (50%)]	Loss: 0.405498
Train Epoch: 15 [4480

Train Epoch: 26 [7040/7664 (92%)]	Loss: 0.399256

Test set: Average loss: 0.3044, Accuracy: 2055/2396 (86%)

Train Epoch: 27 [0/7664 (0%)]	Loss: 0.214623
Train Epoch: 27 [640/7664 (8%)]	Loss: 0.405307
Train Epoch: 27 [1280/7664 (17%)]	Loss: 0.308908
Train Epoch: 27 [1920/7664 (25%)]	Loss: 0.221991
Train Epoch: 27 [2560/7664 (33%)]	Loss: 0.283983
Train Epoch: 27 [3200/7664 (42%)]	Loss: 0.397584
Train Epoch: 27 [3840/7664 (50%)]	Loss: 0.315455
Train Epoch: 27 [4480/7664 (58%)]	Loss: 0.190392
Train Epoch: 27 [5120/7664 (67%)]	Loss: 0.372740
Train Epoch: 27 [5760/7664 (75%)]	Loss: 0.236591
Train Epoch: 27 [6400/7664 (83%)]	Loss: 0.289362
Train Epoch: 27 [7040/7664 (92%)]	Loss: 0.398599

Test set: Average loss: 0.2991, Accuracy: 2059/2396 (86%)

Train Epoch: 28 [0/7664 (0%)]	Loss: 0.207938
Train Epoch: 28 [640/7664 (8%)]	Loss: 0.401619
Train Epoch: 28 [1280/7664 (17%)]	Loss: 0.302255
Train Epoch: 28 [1920/7664 (25%)]	Loss: 0.217803
Train Epoch: 28 [2560/7664 (33%)]	Loss: 0.279563
Train Epoc

Train Epoch: 39 [5120/7664 (67%)]	Loss: 0.304041
Train Epoch: 39 [5760/7664 (75%)]	Loss: 0.182775
Train Epoch: 39 [6400/7664 (83%)]	Loss: 0.246064
Train Epoch: 39 [7040/7664 (92%)]	Loss: 0.396635

Test set: Average loss: 0.2478, Accuracy: 2130/2396 (89%)

Train Epoch: 40 [0/7664 (0%)]	Loss: 0.160138
Train Epoch: 40 [640/7664 (8%)]	Loss: 0.340049
Train Epoch: 40 [1280/7664 (17%)]	Loss: 0.244713
Train Epoch: 40 [1920/7664 (25%)]	Loss: 0.159069
Train Epoch: 40 [2560/7664 (33%)]	Loss: 0.220411
Train Epoch: 40 [3200/7664 (42%)]	Loss: 0.312574
Train Epoch: 40 [3840/7664 (50%)]	Loss: 0.304633
Train Epoch: 40 [4480/7664 (58%)]	Loss: 0.163330
Train Epoch: 40 [5120/7664 (67%)]	Loss: 0.296846
Train Epoch: 40 [5760/7664 (75%)]	Loss: 0.179453
Train Epoch: 40 [6400/7664 (83%)]	Loss: 0.242372
Train Epoch: 40 [7040/7664 (92%)]	Loss: 0.404277

Test set: Average loss: 0.2452, Accuracy: 2136/2396 (89%)

Train Epoch: 41 [0/7664 (0%)]	Loss: 0.158934
Train Epoch: 41 [640/7664 (8%)]	Loss: 0.336056
Train Epoc